# Below is a step-by-step example that how the pharma_subgraph KG used for SQID is built

Set up environment variables with location of the input files

In [13]:
%env kgtk_files=kgtk_files
%env ttl_files=ttl_files
%env ES_DATA_FOLDER_PATH = flask/es/es_data

env: kgtk_files=kgtk_files
env: ttl_files=ttl_files
env: ES_DATA_FOLDER_PATH=flask/es/es_data


## Building Wikidata triples and Mediawiki json files

pharma_subgraph.tsv link: https://drive.google.com/file/d/11oAhqCNR8vAIKXv0IhuIcy-L6NuDFxKF/view?usp=sharing

### Generate Wikidata Triples (under sqid/ttl_files) from KGTK TSV (pharma_subgraph) -- kgtk generate_wikidata_triples

In [2]:
!cat $kgtk_files/pharma_subgraph.tsv \
| kgtk generate_wikidata_triples -pf flask/flask_data/wikidata_properties.tsv -w yes -pd yes > pharma_subgraph.ttl --debug

In [10]:
# stat of output ttl file
!stat pharma_subgraph.ttl

  File: pharma_subgraph.ttl
  Size: 6020115   	Blocks: 11768      IO Block: 4096   regular file
Device: 10305h/66309d	Inode: 280323      Links: 1
Access: (0664/-rw-rw-r--)  Uid: ( 1004/mingyuet)   Gid: ( 1004/mingyuet)
Access: 2020-10-27 00:45:02.076015170 +0000
Modify: 2020-10-27 00:44:28.192014002 +0000
Change: 2020-10-27 00:44:28.192014002 +0000
 Birth: -


In [9]:
!head -n 5 pharma_subgraph.ttl

@prefix wikibase: <http://wikiba.se/ontology#> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix wdt: <http://www.wikidata.org/prop/direct/> .
@prefix wdtn: <http://www.wikidata.org/prop/direct-normalized/> .
@prefix wdno: <http://www.wikidata.org/prop/novalue/> .


In [11]:
!mv *.ttl ttl_files/

### Generate Mediawiki JSON (under sqid/flask/es/es_data/) from KGTK TSV -- kgtk generate-mediawiki-jsons

In [14]:
!cp $kgtk_files/pharma_subgraph.tsv $ES_DATA_FOLDER_PATH/pharma_subgraph.tsv
 
!cat $ES_DATA_FOLDER_PATH/pharma_subgraph.tsv \
| kgtk generate-mediawiki-jsons -pf flask/flask_data/wikidata_properties.tsv -w yes -pd yes

In [17]:
# stats of how many files in ES_DATA_FOLDER_PATH
!cd $ES_DATA_FOLDER_PATH/ && ls | wc -l

34


In [19]:
!mv *.jsonl $ES_DATA_FOLDER_PATH/

## Load trile file into Blazegraph

Given triple files loaded into Blazegraph

## Load json file into ES

### Install all required libraries

In [20]:
!python3 -m venv sqid-env
!source sqid-env/bin/activate

!cd flask && pip install -r requirements.txt
!npm install .

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
npm WARN bootstrap@4.5.2 requires a peer of jquery@1.9.1 - 3 but none is installed. You must install peer dependencies yourself.
npm WARN optional SKIPPING OPTIONAL DEPENDENCY: fsevents@2.1.3 (node_modules/fsevents):
npm WARN notsup SKIPPING OPTIONAL DEPENDENCY: Unsupported platform for fsevents@2.1.3: wanted {"os":"darwin","arch":"any"} (current: {"os":"linux","arch":"x64"})
npm WARN optional SKIPPING OPTIONAL DEPENDENCY: fsevents@1.2.13 (node_modules/jest-haste-map/node_modules/fsevents):
npm WARN notsup SKIPPING OPTIONAL DEPENDENCY: Unsupported platform for fsevents@1.2.13: wanted {"os":"darwin","arch":"any"} (current: {"os":"linux","arch":"x64"})
npm WARN optional SKIPPING OPTIONAL DEPENDENCY: fsevents@1.2.13 (node_modules/webpack-dev-server/node_modules/fsevents):
npm WARN notsup SKIPPING OPTIONAL DEPE

### Modify Configuration file (Global configure)
Parameters:
- HOST = "localhost"
- SQID_PORT = 8051
- SPARQL_PORT = 10002
- FLASK_PORT = 5556
- ES_INDEX = "kgtk_files"
- ELASTICSEARCH_PORT = 9200
- ES_DATA_FOLDER_PATH = "/home/mingyuet/sqid/flask/es/es_data"

In [21]:
!head -n 15 global_config.py

import argparse

parser = argparse.ArgumentParser(description='Enter congiguration parameters')
parser.add_argument('--is_generate', type=bool, help='is_generate', default=False)
args = parser.parse_args()

HOST = "localhost"
SQID_PORT = 8051
SPARQL_PORT = 11102
FLASK_PORT = 5556
ES_INDEX = "kgtk_files"
ELASTICSEARCH_PORT = 9200
is_generate = args.is_generate

# Flask application config


### Create Index for ES

In [31]:
from elasticsearch import Elasticsearch
import os
from global_config import ES_INDEX

TabError: inconsistent use of tabs and spaces in indentation (global_config.py, line 42)

In [29]:
%tb

SystemExit: 2

In [ ]:
es = Elasticsearch()

In [26]:
# index name is defined in global_config file, ES_INDEX, please change this accordingly
if not es.indices.exists(index=ES_INDEX):
    os.system("cd flask/es/ && python create_index.py")

NameError: name 'ES_INDEX' is not defined

### import index data from es_data (mediawiki json files)

In [21]:
# mediawiki json files path(es_data) is defined in global_config file
if not es.indices.exists(index="kgtk_files"):
    !cd flask/es/ && python import_data.py

100%|███████████████████████████████████████████| 90/90 [00:08<00:00, 10.87it/s]


### Run Global Configure File to generate endpoints for SQID and ES

In [24]:
!python global_config.py --is_generate True

### Run ES index system using Flask

and

### Run SQID through npm run serve (development enviorment)

In [5]:
import subprocess
out = subprocess.call("ls -l", shell=True)
out = subprocess.call("cd ..", shell=True)